In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training-dataset/newdataset.csv


In [2]:
train=pd.read_csv("/kaggle/input/training-dataset/newdataset.csv")

In [3]:
train.head

<bound method NDFrame.head of        length_url  domain_length  domain_in_ip  server_client_domain  \
0              25             17             0                     0   
1             223             16             0                     0   
2              15             14             0                     0   
3              81             19             0                     0   
4              19             19             0                     0   
...           ...            ...           ...                   ...   
88642          23             23             0                     0   
88643          34             34             0                     0   
88644          70             22             0                     0   
88645          28             27             0                     0   
88646          16             16             0                     0   

       directory_length  file_length  params_length  tld_present_params  \
0                     8       

In [4]:
train = train.drop('time_domain_expiration', axis=1)
train = train.drop('time_domain_activation', axis=1)
train = train.drop('directory_length',axis=1)


In [5]:
train.columns

Index(['length_url', 'domain_length', 'domain_in_ip', 'server_client_domain',
       'file_length', 'params_length', 'tld_present_params', 'email_in_url',
       'time_response', 'asn_ip', 'domain_google_index', 'url_shortened',
       'phishing', 'qty_char_domain'],
      dtype='object')

In [6]:
#Splitting Datasets
data = train.drop('phishing', axis=1)
label = train['phishing']

In [7]:
#finding euclidian distance
'''def calculate_euclidean_distances(dataset, real_time_url_features):
    euclidean_distances = np.linalg.norm(newdata.iloc[:, :-1].values - real_time_url_features, axis=1)
    return euclidean_distances'''

    

'def calculate_euclidean_distances(dataset, real_time_url_features):\n    euclidean_distances = np.linalg.norm(newdata.iloc[:, :-1].values - real_time_url_features, axis=1)\n    return euclidean_distances'

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data,label,test_size=0.3, random_state=42)

In [9]:
def display_test_scores(test, pred):
    str_out = ""
    str_out += ("TEST SCORES")
    str_out += ("\n")

    #print accuracy
    accuracy = accuracy_score(test, pred)
    str_out += ("ACCURACY: {:.4f}\n".format(accuracy))
    str_out += ("\n")

    #print AUC score
    auc = roc_auc_score(test, pred)
    str_out += ("AUC: {:.4f}\n".format(auc))
    str_out += ("\n")

    #print confusion matrix
    str_out += ("CONFUSION MATRIX:\n")
    conf_mat = confusion_matrix(test, pred)
    str_out += ("{}".format(conf_mat))
    str_out += ("\n")
    str_out += ("\n")

    #print FP, FN
    str_out += ("FALSE POSITIVES:\n")
    fp = conf_mat[1][0]
    pos_labels = conf_mat[1][0]+conf_mat[1][1]
    str_out += ("{} out of {} positive labels ({:.4f}%)\n".format(fp, pos_labels,fp/pos_labels))
    str_out += ("\n")

    str_out += ("FALSE NEGATIVES:\n")
    fn = conf_mat[0][1]
    neg_labels = conf_mat[0][1]+conf_mat[0][0]
    str_out += ("{} out of {} negative labels ({:.4f}%)\n".format(fn, neg_labels, fn/neg_labels))
    str_out += ("\n")

    #print classification report
    str_out += ("PRECISION, RECALL, F1 scores:\n")
    str_out += ("{}".format(classification_report(test, pred)))
    
    false_indexes = np.where(test != pred)
    return str_out, false_indexes

In [10]:
from sklearn.neighbors import KNeighborsClassifier
import pickle as pic
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, classification_report, roc_auc_score
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)
pic.dump(knn,open('knn.pkl','wb'))
prediction = knn.predict(X_val)
results,false = display_test_scores(y_val, prediction)
print(results)

TEST SCORES
ACCURACY: 0.9201

AUC: 0.8947

CONFUSION MATRIX:
[[17029   420]
 [ 1706  7440]]

FALSE POSITIVES:
1706 out of 9146 positive labels (0.1865%)

FALSE NEGATIVES:
420 out of 17449 negative labels (0.0241%)

PRECISION, RECALL, F1 scores:
              precision    recall  f1-score   support

           0       0.91      0.98      0.94     17449
           1       0.95      0.81      0.87      9146

    accuracy                           0.92     26595
   macro avg       0.93      0.89      0.91     26595
weighted avg       0.92      0.92      0.92     26595



In [11]:
from sklearn.metrics import mean_squared_error

y_train_pred = knn.predict(X_train)
mse_train = mean_squared_error(y_train, y_train_pred)
mse_val = mean_squared_error(y_val, prediction)

bias = mse_train
variance = mse_val - mse_train

print("Bias (Training MSE):", bias)
print("Variance:", variance)


Bias (Training MSE): 0.042416038161541936
Variance: 0.03752380015393089


In [12]:
from sklearn.neural_network import MLPClassifier
NN = MLPClassifier()
NN.fit(X_train,y_train)
prediction = NN.predict(X_val)
results,false = display_test_scores(y_val, prediction)
print(results)

TEST SCORES
ACCURACY: 0.7620

AUC: 0.6600

CONFUSION MATRIX:
[[17217   232]
 [ 6098  3048]]

FALSE POSITIVES:
6098 out of 9146 positive labels (0.6667%)

FALSE NEGATIVES:
232 out of 17449 negative labels (0.0133%)

PRECISION, RECALL, F1 scores:
              precision    recall  f1-score   support

           0       0.74      0.99      0.84     17449
           1       0.93      0.33      0.49      9146

    accuracy                           0.76     26595
   macro avg       0.83      0.66      0.67     26595
weighted avg       0.80      0.76      0.72     26595



In [24]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu',input_shape=(13,) ),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=200, batch_size=32)
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation accuracy: {val_accuracy * 100:.2f}%")


Epoch 1/200
1940/1940 [==============================] - 4s 2ms/step - loss: 52.8350 - accuracy: 0.6266
Epoch 2/200
1940/1940 [==============================] - 3s 2ms/step - loss: 34.7305 - accuracy: 0.6787
Epoch 3/200
1940/1940 [==============================] - 3s 2ms/step - loss: 20.5316 - accuracy: 0.7144
Epoch 4/200
1940/1940 [==============================] - 3s 2ms/step - loss: 16.0716 - accuracy: 0.7315
Epoch 5/200
1940/1940 [==============================] - 3s 2ms/step - loss: 12.5727 - accuracy: 0.7410
Epoch 6/200
1940/1940 [==============================] - 4s 2ms/step - loss: 8.1386 - accuracy: 0.7486
Epoch 7/200
1940/1940 [==============================] - 3s 2ms/step - loss: 5.7922 - accuracy: 0.7663
Epoch 8/200
1940/1940 [==============================] - 3s 2ms/step - loss: 4.5581 - accuracy: 0.7652
Epoch 9/200
1940/1940 [==============================] - 3s 2ms/step - loss: 2.5906 - accuracy: 0.7848
Epoch 10/200
1940/1940 [==============================] - 4s 2ms/ste